## Investment

In [55]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [38]:
targets = {
    'AMZN': 0.40, # Amazon
    'CSCO': 0.30, # Cisco
    'GE': 0.30 # GE
}

portfolio = pd.DataFrame(
    index=list(targets.keys()) + ['CASH'],
    data={
        'date': '2018-01-01',
        'price': [np.NaN, np.NaN, np.NaN, 1],
        'target': [0.4, 0.3, 0.3, 0],
        'allocation': [0, 0, 0, 1],
        'shares': [0, 0, 0, 10000],
        'market_value': [0, 0, 0, 10000]
    }
)

print(portfolio)

      allocation        date  market_value  price  shares  target
AMZN           0  2018-01-01             0    NaN       0     0.4
CSCO           0  2018-01-01             0    NaN       0     0.3
GE             0  2018-01-01             0    NaN       0     0.3
CASH           1  2018-01-01         10000    1.0   10000     0.0


In [39]:
def instantiate_portfolio(targets, starting_balance):
    targets['CASH'] = 0
    tickers = list(targets.keys())
    df = pd.DataFrame(
        index=tickers,
        columns=[
            'date', 'price', 'target',
            'allocation', 'shares', 'market_value'
        ]
    )
    
    df.shares = 0
    df.market_value = 0
    df.allocation = 0
    df.update(
        pd.DataFrame
            .from_dict(targets, orient='index')
            .rename(columns={0: 'target'})
    )
    
    df.at['CASH', 'shares'] = starting_balance
    
    return df

In [82]:
portfolio = instantiate_portfolio(
    {
        'AMZN': 0.40, # Amazon
        'CSCO': 0.30, # Cisco
        'GE': 0.30 # GE
    },
    50000
)

In [83]:
print(portfolio)

     date price target  allocation  shares  market_value
AMZN  NaN   NaN    0.4           0       0             0
CSCO  NaN   NaN    0.3           0       0             0
GE    NaN   NaN    0.3           0       0             0
CASH  NaN   NaN      0           0   50000             0


In [42]:
print(targets)

{'AMZN': 0.4, 'CSCO': 0.3, 'GE': 0.3}


In [43]:
list(targets.keys())

['AMZN', 'CSCO', 'GE']

In [44]:
print(pd.DataFrame.from_dict(targets, orient='index'))


        0
AMZN  0.4
CSCO  0.3
GE    0.3


In [45]:
def update_prices(portfolio, prices):
    prices['CASH'] = 1
    portfolio.update(pd.DataFrame({'price': prices}))
    portfolio.date = prices.name
    portfolio.market_value = portfolio.shares * portfolio.price


In [46]:
prices = pd.Series(
    name='2018-01-01',
    data={'AMZN': 945.21,'CSCO': 30.52, 'GE': 29.27}
)
print(prices)
update_prices(portfolio, prices)

AMZN    945.21
CSCO     30.52
GE       29.27
Name: 2018-01-01, dtype: float64


In [47]:
print(portfolio)

            date   price target  allocation  shares market_value
AMZN  2018-01-01  945.21    0.4           0       0            0
CSCO  2018-01-01   30.52    0.3           0       0            0
GE    2018-01-01   29.27    0.3           0       0            0
CASH  2018-01-01       1      0           0   10000        10000


In [48]:
def get_order(portfolio):
    total_value = portfolio.market_value.sum()
    order = (
            (total_value * portfolio.target // portfolio.price)
            - portfolio.shares
    ).drop('CASH')
    return order

def deposit(portfolio, amount):
    portfolio.at['CASH', 'shares'] += amount
    portfolio.at['CASH', 'market_value'] = portfolio.at['CASH', 'shares']

In [49]:
deposit(portfolio, 1000)

In [50]:
order = get_order(portfolio)
print(order)

AMZN      4
CSCO    108
GE      112
dtype: object


In [51]:
def simulate_process_order(portfolio, order):
    starting_cash = portfolio.at['CASH', 'shares']
    cash_adjustment = np.sum(order * portfolio.price)
    portfolio.shares += order
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.at['CASH', 'shares'] = starting_cash - cash_adjustment
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.allocation = (
        portfolio.market_value / portfolio.market_value.sum()
    )


In [52]:
simulate_process_order(portfolio, order)
print(portfolio)

            date   price target allocation  shares market_value
AMZN  2018-01-01  945.21    0.4   0.343713       4      3780.84
CSCO  2018-01-01   30.52    0.3   0.299651     108      3296.16
GE    2018-01-01   29.27    0.3   0.298022     112      3278.24
CASH  2018-01-01       1      0  0.0586145  644.76       644.76


Working with API for extract Stock Quotes

In [56]:
API_KEY = os.environ.get('AV_KEY')
TODAY = pd.Timestamp.today().normalize()

In [57]:
def get_price(ticker, outputsize='compact', most_recent=False):
    URL = 'https://www.alphavantage.co/query?'
    payload = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': ticker,
        'apikey': API_KEY,
        'outputsize': outputsize
    }
    
    r = requests.get(URL, params=payload)
    p = pd.DataFrame(r.json()['Time Series (Daily)']).T['4. close']
    df = pd.DataFrame({ticker: p.apply(float)})
    df.index = pd.to_datetime(df.index)
    if most_recent:
        return df.tail(1)
    return df

In [59]:
print(get_price('AMZN').tail()[:10])

               AMZN
2018-04-12  1448.50
2018-04-13  1430.79
2018-04-16  1441.50
2018-04-17  1503.83
2018-04-18  1527.84


In [66]:
def get_historical(tickers, start_date, end_date):
    df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='D'))
    for t in tickers:
        df = pd.concat([
            df,
            get_price(t, outputsize='full')],
        axis=1,
        join_axes=[df.index]
        )
    df = df.fillna(method='ffill').dropna()
    return df

In [67]:
historical_prices = get_historical(
    tickers=['AMZN', 'CSCO', 'GE'],
    start_date=pd.Timestamp(2016, 1, 1),
    end_date=TODAY
)

In [68]:
print(historical_prices.tail(10))

               AMZN   CSCO     GE
2018-04-09  1406.08  41.17  12.83
2018-04-10  1436.22  42.51  13.05
2018-04-11  1427.05  42.43  12.97
2018-04-12  1448.50  43.34  13.18
2018-04-13  1430.79  43.00  13.49
2018-04-14  1430.79  43.00  13.49
2018-04-15  1430.79  43.00  13.49
2018-04-16  1441.50  43.30  13.33
2018-04-17  1503.83  44.59  13.79
2018-04-18  1503.83  44.59  13.79


In [86]:
prices = historical_prices.loc['2016-01-24']
prices

AMZN    596.38
CSCO     23.37
GE       28.24
Name: 2016-01-24 00:00:00, dtype: float64

In [90]:
portafolio = instantiate_portfolio(targets, 100000.00)
prices = historical_prices.loc['2017-06-06']
update_prices(portafolio, prices)
order = get_order(portafolio)
simulate_process_order(portafolio, order)
portafolio.market_value.sum()

100000.0

In [91]:
print(portafolio)


           date  price target allocation  shares market_value
AMZN 2017-06-06   1003    0.4    0.39117      39        39117
CSCO 2017-06-06  31.56    0.3    0.29982     950        29982
GE   2017-06-06  27.93    0.3   0.299968    1074      29996.8
CASH 2017-06-06      1      0  0.0090418  904.18       904.18


In [97]:
dates = pd.date_range('2017-01-01', '2017-12-31', freq='Q').tolist()
for d in dates:
    prices = historical_prices.loc[d]
    update_prices(portafolio, prices)
    order = get_order(portafolio)
    print(f'{d}:\n{order}')
    simulate_process_order(portafolio, order)
portafolio.market_value.sum()

2017-03-31 00:00:00:
AMZN     5
CSCO   -67
GE     -72
dtype: object
2017-06-30 00:00:00:
AMZN    -4
CSCO    57
GE      88
dtype: object
2017-09-30 00:00:00:
AMZN      0
CSCO    -75
GE      113
dtype: object
2017-12-31 00:00:00:
AMZN     -6
CSCO    -72
GE      538
dtype: object


101283.26

In [99]:
print(portafolio)

           date    price target  allocation  shares market_value
AMZN 2017-12-31  1169.47    0.4    0.392582      34        39762
CSCO 2017-12-31     38.3    0.3    0.299871     793      30371.9
GE   2017-12-31    17.45    0.3    0.299955    1741      30380.4
CASH 2017-12-31        1      0  0.00759188  768.93       768.93
